In [41]:
# Importación de paquetes requeridos
import warnings

import io
import requests
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
plt.close("all")
import scipy
import numpy as np

from IPython.display import display, Markdown, Latex

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import dateparser
import json

In [42]:
# Función auxiliar para visualizar los datos
def pp(obj):
    print(json.dumps(obj, indent=2))

In [48]:
# Parmámetros
iga_api_base_url = 'http://ing-acc-movil01/ingenier@/symfony/public/index.php/api'
site_names = ['A04-S001', 'A06-S001', 'A01-S001']
site_names_str = ', '.join(site_names)

In [83]:
display(Markdown(f"# Ejemplo 002: Descarga del reporte de Datos Outdoor para un conjunto de sitios en formato Excel"))

# Ejemplo 002: Descarga del reporte de Datos Outdoor para un conjunto de sitios en formato Excel

In [50]:
# Llamamos a un endpoint de prueba en la API de IGA para verificar conectividad
display(Markdown("### IGA API test"))
x = requests.get(iga_api_base_url + '/public/db-sync-state')
pp(x.json())

### IGA API test

{
  "status": "success",
  "data": {
    "Uid": "41158",
    "State": "DB_SYNC_STATE_IDLE",
    "LastSyncTimestamp": "16-05-2022 16:24:17",
    "SyncState": "OK",
    "ProjectsUpdated": "DataManager, NORAN"
  }
}


In [51]:
# Llamamos al endpoint generic-report para obtener el reporte de datos outdoor del sitio seleccionado:

# Parámetros en el payload de la request con que llamamos al endpoint de IGA:
payload = {
    # Nombre del reporte: datos_outdoor
    "reportName": "datos_outdoor",
    # Sitios seleccionados: los códigos de sitio que están almacenados en la variable site_names
    "siteNames": site_names,
    # Parámetros de filtrado: ninguno (objeto vacío: {})
    "filteringParams": {},
    # Seleccionar por nombres de sitio: sí
    "selectBySiteNames": True,
    # Seleccionar por parámetros de filtrado: no
    "selectByFilteringParams": False
}
x = requests.post(iga_api_base_url + '/public/report/generic-report', json=payload)

# En x almacenamos los datos devueltos por el endpoint de IGA. Es un archivo xlsx en formato binario

# Usando pandas, con la función read_excel, leemos los datos y los almacenamos en memoria como un objeto de python:
with io.BytesIO(x.content) as fh:
    df = pd.read_excel(fh)


In [52]:
# Visualizamos los datos obtenidos:
display(Markdown(f"# Datos Outdoor - Sitios: {site_names_str}"))
display(Markdown(f"### Tabla con orientación normal:"))
display(df)

display(Markdown(f"### Tabla transpuesta:"))
# Los mismos datos pero transponiéndolos (intercambiando filas por columnas) para poder ver mejor todos los nombres de columnas:
display(df.transpose())

# Datos Outdoor - Sitios: A04-S001, A06-S001, A01-S001

### Tabla con orientación normal:

,Proyecto,Código,Sector,Portadora,Estado,Latitud Antena,Longitud Antena,Id Antena,Modelo Antena,Id Puerto,...,Altura Antena,Azimuth Físico,Azimuth Efectivo,Tilt Mec.,Tilt Eléc.,Twist,Ancho Horiz.,Ancho Vert.,Ganancia dBi,Cota de Terreno
0,MEDI,A01-S001,XDCAMPB11,LTE FDD_700MHz_9312_NB_IoT,No Operativo,-34.377472,-64.506528,19,ATR4518R13,3,...,40.0,40.0,40.0,-0.0,2.0,0.0,65,13,14.220000,230.69
1,MEDI,A01-S001,XDCAMPB12,LTE FDD_700MHz_9312_NB_IoT,No Operativo,-34.377472,-64.506528,20,ATR4518R13,3,...,40.0,110.0,110.0,-0.0,2.0,0.0,65,13,14.220000,230.69
2,MEDI,A01-S001,XDCAMPB13,LTE FDD_700MHz_9312_NB_IoT,No Operativo,-34.377472,-64.506528,21,ATR4518R13,3,...,40.0,280.0,280.0,-0.0,2.0,0.0,65,13,14.220000,230.69
3,MEDI,A01-S001,XDCAMPG11,GSM1900,No Operativo,-34.377472,-64.506528,1,ADU4517R3,3,...,44.0,40.0,41.0,-0.0,5.0,0.0,64,6,16.826250,230.69
4,MEDI,A01-S001,XDCAMPG12,GSM1900,No Operativo,-34.377472,-64.506528,2,ADU4517R3,3,...,44.0,110.0,111.0,-0.0,3.0,0.0,64,6,16.826250,230.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,LISU,A06-S001,XACHIPV12,WCDMA850_4358,Operativo,-30.960312,-62.338604,3,DBXLH-6565A-VTM,1,...,56.0,120.0,120.0,-0.0,1.0,0.0,67,15,13.875438,-999.00
92,LISU,A06-S001,XACHIPV13,WCDMA850_4358,Operativo,-30.960312,-62.338604,4,DBXLH-6565A-VTM,1,...,56.0,240.0,240.0,-0.0,1.0,0.0,67,15,13.875438,-999.00
93,LISU,A06-S001,XACHIPV21,WCDMA850_4379,Operativo,-30.960312,-62.338604,2,DBXLH-6565A-VTM,1,...,56.0,0.0,0.0,-0.0,1.0,0.0,67,15,13.875438,-999.00
94,LISU,A06-S001,XACHIPV22,WCDMA850_4379,Operativo,-30.960312,-62.338604,3,DBXLH-6565A-VTM,1,...,56.0,120.0,120.0,-0.0,1.0,0.0,67,15,13.875438,-999.00


### Tabla transpuesta:

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
Proyecto,MEDI,MEDI,MEDI,MEDI,MEDI,MEDI,MEDI,MEDI,MEDI,MEDI,...,LISU,LISU,LISU,LISU,LISU,LISU,LISU,LISU,LISU,LISU
Código,A01-S001,A01-S001,A01-S001,A01-S001,A01-S001,A01-S001,A01-S001,A01-S001,A01-S001,A01-S001,...,A06-S001,A06-S001,A06-S001,A06-S001,A06-S001,A06-S001,A06-S001,A06-S001,A06-S001,A06-S001
Sector,XDCAMPB11,XDCAMPB12,XDCAMPB13,XDCAMPG11,XDCAMPG12,XDCAMPG13,XDCAMPH11,XDCAMPH12,XDCAMPH13,XDCAMPL11,...,XACHIPN13,XACHIPO11,XACHIPO12,XACHIPO13,XACHIPV11,XACHIPV12,XACHIPV13,XACHIPV21,XACHIPV22,XACHIPV23
Portadora,LTE FDD_700MHz_9312_NB_IoT,LTE FDD_700MHz_9312_NB_IoT,LTE FDD_700MHz_9312_NB_IoT,GSM1900,GSM1900,GSM1900,GSM850,GSM850,GSM850,LTE FDD_AWS_2225,...,LTE FDD_700MHz_9360,LTE FDD_850MHz_2476,LTE FDD_850MHz_2476,LTE FDD_850MHz_2476,WCDMA850_4358,WCDMA850_4358,WCDMA850_4358,WCDMA850_4379,WCDMA850_4379,WCDMA850_4379
Estado,No Operativo,No Operativo,No Operativo,No Operativo,No Operativo,No Operativo,Operativo,Operativo,Operativo,Operativo,...,Operativo,No Operativo,No Operativo,No Operativo,Operativo,Operativo,Operativo,Operativo,Operativo,Operativo
Latitud Antena,-34.377472,-34.377472,-34.377472,-34.377472,-34.377472,-34.377472,-34.377472,-34.377472,-34.377472,-34.377472,...,-30.960312,-30.960312,-30.960312,-30.960312,-30.960312,-30.960312,-30.960312,-30.960312,-30.960312,-30.960312
Longitud Antena,-64.506528,-64.506528,-64.506528,-64.506528,-64.506528,-64.506528,-64.506528,-64.506528,-64.506528,-64.506528,...,-62.338604,-62.338604,-62.338604,-62.338604,-62.338604,-62.338604,-62.338604,-62.338604,-62.338604,-62.338604
Id Antena,19,20,21,1,2,3,1,2,3,19,...,7,2,3,4,2,3,4,2,3,4
Modelo Antena,ATR4518R13,ATR4518R13,ATR4518R13,ADU4517R3,ADU4517R3,ADU4517R3,ADU4517R3,ADU4517R3,ADU4517R3,ATR4518R13,...,ATR4518R13,DBXLH-6565A-VTM,DBXLH-6565A-VTM,DBXLH-6565A-VTM,DBXLH-6565A-VTM,DBXLH-6565A-VTM,DBXLH-6565A-VTM,DBXLH-6565A-VTM,DBXLH-6565A-VTM,DBXLH-6565A-VTM
Id Puerto,3,3,3,3,3,3,1,1,1,1,...,3,1,1,1,1,1,1,1,1,1


In [61]:
# Aplicamos filtros
# Seleccionamos los datos únicamente del primer sitio
site_name = site_names[0]
display(Markdown(f"### Sitio seleccionado: {site_name}"))
df1 = df.loc[df['Código'] == site_name]
display(df1)

### Sitio seleccionado: A04-S001

,Proyecto,Código,Sector,Portadora,Estado,Latitud Antena,Longitud Antena,Id Antena,Modelo Antena,Id Puerto,...,Altura Antena,Azimuth Físico,Azimuth Efectivo,Tilt Mec.,Tilt Eléc.,Twist,Ancho Horiz.,Ancho Vert.,Ganancia dBi,Cota de Terreno
33,LISU,A04-S001,BBARADB11,LTE FDD_700MHz_9312_NB_IoT,Operativo,-33.805919,-59.50455,4,ODI-065R15MM18JJJ02-GQ,9,...,44.0,100.0,100.0,3.0,6.0,0.0,58,14,14.404444,29.32
34,LISU,A04-S001,BBARADB12,LTE FDD_700MHz_9312_NB_IoT,Operativo,-33.805919,-59.50455,5,ODI-065R15MM18JJJ02-GQ,9,...,44.0,220.0,220.0,2.0,6.0,0.0,58,14,14.404444,29.32
35,LISU,A04-S001,BBARADB13,LTE FDD_700MHz_9312_NB_IoT,Operativo,-33.805919,-59.50455,6,ODI-065R15MM18JJJ02-GQ,9,...,44.0,340.0,340.0,2.0,5.0,0.0,58,14,14.404444,29.32
36,LISU,A04-S001,BBARADF11,LTE FDD_2600MHz_3150,Operativo,-33.805919,-59.50455,4,ODI-065R15MM18JJJ02-GQ,1,...,44.0,100.0,100.0,3.0,2.0,0.0,60,6,17.281273,29.32
37,LISU,A04-S001,BBARADF12,LTE FDD_2600MHz_3150,Operativo,-33.805919,-59.50455,5,ODI-065R15MM18JJJ02-GQ,1,...,44.0,220.0,220.0,2.0,5.0,0.0,60,6,17.281273,29.32
38,LISU,A04-S001,BBARADF13,LTE FDD_2600MHz_3150,Operativo,-33.805919,-59.50455,6,ODI-065R15MM18JJJ02-GQ,1,...,44.0,340.0,340.0,2.0,5.0,0.0,60,6,17.281273,29.32
39,LISU,A04-S001,BBARADG11,GSM1900,Operativo,-33.805919,-59.50455,4,ODI-065R15MM18JJJ02-GQ,1,...,44.0,100.0,100.0,3.0,2.0,0.0,60,6,17.281273,29.32
40,LISU,A04-S001,BBARADG12,GSM1900,Operativo,-33.805919,-59.50455,5,ODI-065R15MM18JJJ02-GQ,1,...,44.0,220.0,220.0,2.0,5.0,0.0,60,6,17.281273,29.32
41,LISU,A04-S001,BBARADG13,GSM1900,Operativo,-33.805919,-59.50455,6,ODI-065R15MM18JJJ02-GQ,1,...,44.0,340.0,340.0,2.0,5.0,0.0,60,6,17.281273,29.32
42,LISU,A04-S001,BBARADH11,GSM850,Operativo,-33.805919,-59.50455,4,ODI-065R15MM18JJJ02-GQ,7,...,44.0,100.0,102.0,3.0,6.0,0.0,56,14,14.376444,29.32


In [82]:
# Aplicamos un nuevo filtro para seleccionar únicamente las celdas de una portadora:
portadora = 'WCDMA850_4379'
display(Markdown(f"### Sitio seleccionado: {site_name}, portadora seleccionada: {portadora}"))
df2 = df1.loc[df1['Portadora'] == portadora]
display(df2)

# Los mismos datos, pero ordenando por nombre de celda en orden inverso
display(Markdown(f"### Sitio seleccionado: {site_name}, portadora seleccionada: {portadora}, ordenado por nombre de celda (desc)"))
display(df2.sort_values(by='Sector', ascending=False))

### Sitio seleccionado: A04-S001, portadora seleccionada: WCDMA850_4379

,Proyecto,Código,Sector,Portadora,Estado,Latitud Antena,Longitud Antena,Id Antena,Modelo Antena,Id Puerto,...,Altura Antena,Azimuth Físico,Azimuth Efectivo,Tilt Mec.,Tilt Eléc.,Twist,Ancho Horiz.,Ancho Vert.,Ganancia dBi,Cota de Terreno
63,LISU,A04-S001,BBARADV21,WCDMA850_4379,Operativo,-33.805919,-59.50455,7,2UNPX203.6R2,1,...,38.0,100.0,70.0,-0.0,3.0,0.0,41,24,13.427500,29.0
64,LISU,A04-S001,BBARADV22,WCDMA850_4379,Operativo,-33.805919,-59.50455,8,2UNPX203.6R2,1,...,38.0,220.0,190.0,-0.0,3.0,0.0,41,24,13.427500,29.0
65,LISU,A04-S001,BBARADV23,WCDMA850_4379,Operativo,-33.805919,-59.50455,9,2UNPX203.6R2,1,...,38.0,340.0,310.0,-0.0,8.0,0.0,41,24,13.427500,29.0
66,LISU,A04-S001,BBARADV24,WCDMA850_4379,Operativo,-33.805919,-59.50455,7,2UNPX203.6R2,3,...,38.0,100.0,132.0,-0.0,3.0,0.0,42,24,13.436875,29.0
67,LISU,A04-S001,BBARADV25,WCDMA850_4379,Operativo,-33.805919,-59.50455,8,2UNPX203.6R2,3,...,38.0,220.0,252.0,-0.0,4.0,0.0,42,24,13.436875,29.0
68,LISU,A04-S001,BBARADV26,WCDMA850_4379,Operativo,-33.805919,-59.50455,9,2UNPX203.6R2,3,...,38.0,340.0,12.0,-0.0,6.0,0.0,42,24,13.436875,29.0


### Sitio seleccionado: A04-S001, portadora seleccionada: WCDMA850_4379, ordenado por nombre de celda (desc)

,Proyecto,Código,Sector,Portadora,Estado,Latitud Antena,Longitud Antena,Id Antena,Modelo Antena,Id Puerto,...,Altura Antena,Azimuth Físico,Azimuth Efectivo,Tilt Mec.,Tilt Eléc.,Twist,Ancho Horiz.,Ancho Vert.,Ganancia dBi,Cota de Terreno
68,LISU,A04-S001,BBARADV26,WCDMA850_4379,Operativo,-33.805919,-59.50455,9,2UNPX203.6R2,3,...,38.0,340.0,12.0,-0.0,6.0,0.0,42,24,13.436875,29.0
67,LISU,A04-S001,BBARADV25,WCDMA850_4379,Operativo,-33.805919,-59.50455,8,2UNPX203.6R2,3,...,38.0,220.0,252.0,-0.0,4.0,0.0,42,24,13.436875,29.0
66,LISU,A04-S001,BBARADV24,WCDMA850_4379,Operativo,-33.805919,-59.50455,7,2UNPX203.6R2,3,...,38.0,100.0,132.0,-0.0,3.0,0.0,42,24,13.436875,29.0
65,LISU,A04-S001,BBARADV23,WCDMA850_4379,Operativo,-33.805919,-59.50455,9,2UNPX203.6R2,1,...,38.0,340.0,310.0,-0.0,8.0,0.0,41,24,13.427500,29.0
64,LISU,A04-S001,BBARADV22,WCDMA850_4379,Operativo,-33.805919,-59.50455,8,2UNPX203.6R2,1,...,38.0,220.0,190.0,-0.0,3.0,0.0,41,24,13.427500,29.0
63,LISU,A04-S001,BBARADV21,WCDMA850_4379,Operativo,-33.805919,-59.50455,7,2UNPX203.6R2,1,...,38.0,100.0,70.0,-0.0,3.0,0.0,41,24,13.427500,29.0


In [74]:
# Trabajando con los datos así filtrados, queremos obtener, de la columna Tilt Eléc los valores mínimo, máximo y promedio:
df_tilt_elec = df2['Tilt Eléc.']
pp(list(df_tilt_elec))
# Mínimo
tilt_elec_min = min(df_tilt_elec)
# Máximo
tilt_elec_max = max(df_tilt_elec)
# Promedio
tilt_elec_avg = sum(df_tilt_elec) / len(df_tilt_elec)

display(Markdown(f"#### Tilt Eléc. Mín: {tilt_elec_min}"))
display(Markdown(f"#### Tilt Eléc. Máx: {tilt_elec_max}"))
display(Markdown(f"#### Tilt Eléc. Avg: {tilt_elec_avg}"))

[
  3.0,
  3.0,
  8.0,
  3.0,
  4.0,
  6.0
]


#### Tilt Eléc. Mín: 3.0

#### Tilt Eléc. Máx: 8.0

#### Tilt Eléc. Avg: 4.5